In [1]:
#!conda install -y llama_index
#!conda  install llama_index

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import torch
from string import Template
from pathlib import Path
import json
from tqdm.auto import tqdm
import warnings
warnings.simplefilter("ignore")

import torch

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import torch
from string import Template
from pathlib import Path
import json
from tqdm.auto import tqdm
import warnings
warnings.simplefilter("ignore")

import torch

In [4]:
with open("/kaggle/input/telecom-networks-5g-data/TeleQnA_training.txt", "r") as f:
    train_data = f.read()
    
with open("/kaggle/input/telecom-networks-5g-data/TeleQnA_testing1.txt", "r") as f:
    test_data = f.read()

In [5]:
test_data=json.loads(test_data)
train_data=json.loads(train_data)

In [6]:
def dict_to_frame(text):
    data = []
    for question_id in list(text.keys()):
        question_details=text[question_id]
        data.append({
           "question_id": question_id,
           "question": question_details["question"],
           "option_1": question_details["option 1"],
           "option_2": question_details["option 2"],
           "option_3": question_details.get("option 3", None),
           "option_4": question_details.get("option 4", None),
           "category": question_details["category"],
         })
    return pd.DataFrame(data)

test_df=dict_to_frame(test_data)
train_df=dict_to_frame(train_data)
test_df['question_id']=test_df['question_id'].str.replace('question ', '').astype(int)
train_df

,question_id,question,option_1,option_2,option_3,option_4,category
0,question 0,What is the purpose of the Nmfaf_3daDataManage...,To configure the MFAF to map data or analytics...,To configure the MFAF to stop mapping data or ...,To supply data or analytics from the MFAF to n...,To fetch data or analytics from the MFAF based...,Standards specifications
1,question 4,How does a supporting UE attach to the same co...,It requests the core network node to remember ...,It uses information stored in the UE when it w...,It relies on the SIM/USIM card for information.,It performs a fresh attach procedure.,Standards specifications
2,question 11,When can the setting of the Privacy exception ...,Never,Only during emergency services,Anytime,Only with operator permission,Standards specifications
3,question 12,What does the NEF notify to the AF after deter...,AF Identifier,EAS address information,DNN,DNAI(s) or the updated DNAI information,Standards specifications
4,question 14,"In online charging, how are chargeable events ...",By the CTF,By the OCF,By the CGF,By the CHF,Standards specifications
...,...,...,...,...,...,...,...
1456,question 8111,What services does the UMTS Bearer Service con...,Radio Access Bearer Service and Core Network B...,Te/MT Local Bearer Service and External Bearer...,GERAN and UTRAN FDD/TDD,Iu and Gb Bearer Service,Standards specifications
1457,question 8114,What does the UE provide to the AS for slice a...,Slice-Maximum Bit Rate,S-NSSAI,NSSRG,S-MBR,Standards overview
1458,question 8117,What is the interfering signal for the dynamic...,An E-UTRA signal,A CW signal,An AWGN signal,An interfering signal with a positive offset,Standards specifications
1459,question 8123,What is the purpose of AKMA TLS protocol profi...,To provide secure TLS connections between the ...,To establish a secure tunnel between the netwo...,To generate keys for use in the 5G core,To support non-seamless WLAN offload authentic...,Standards overview


In [7]:
preamble = \
   'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
template = Template('$preamble\n\n$prompt\n\nA) $a\nB) $b\nC) $c\nD) $d')

In [8]:
def format_input(df, idx):
    
    prompt = df.loc[idx, 'question']
    a = df.loc[idx, 'option_1']
    b = df.loc[idx, 'option_2']
    c = df.loc[idx, 'option_3']
    d = df.loc[idx, 'option_4']
   

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d)
    
    return input_text

In [9]:
print(format_input(test_df, 3))

Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice

What type of subcarrier spacing requirements does an NB-IoT Base Station support? [3GPP Release 18]

A) 15 kHz subcarrier spacing requirements only
B) 3.75 kHz subcarrier spacing requirements only
C) Both 15 kHz and 3.75 kHz subcarrier spacing requirements
D) No subcarrier spacing requirements


In [10]:
ss=pd.read_csv('/kaggle/input/telecom-networks-5g-data/SampleSubmission (9).csv')
ans=pd.read_csv('/kaggle/input/telecom-networks-5g-data/Q_A_ID_training.csv')

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
def extract_answer(model_output):
    lines = model_output.splitlines()
    for line in lines:
        try:
            if line.startswith("Answer:"):
                return line.split()[1]  # Split by space and return the second word (answer choice)
        except:
            return 'D'  # Answer not found

In [13]:
if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

In [14]:
def get_ans():
    ans_lst=[]
    for i in tqdm(range(test_df.shape[0])):
        inputs = tokenizer(format_input(test_df, i), return_tensors="pt", return_attention_mask=False).to(device)
        outputs = model.generate(**inputs,pad_token_id=tokenizer.pad_token_id, max_length=200)
        text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        ans=extract_answer(text)[0]
        ans_lst.append(ans)
    return ans_lst
ans_lst=get_ans()


  0%|          | 0/366 [00:00<?, ?it/s]

In [15]:
map_ans={'A':1, 'B':2, 'C':3, 'D':4}

In [16]:
sub=pd.read_csv('/kaggle/input/telecom-networks-5g-data/sub (10).csv')

In [17]:
sub=pd.DataFrame({'Question_ID':test_df['question_id'].values, 'Answer_ID':ans_lst})
sub['Answer_ID']=sub['Answer_ID'].map(map_ans)
sub['Answer_ID']=sub['Answer_ID'].fillna(3).astype(int)
subzindi=pd.concat([sub, ss[~ss['Question_ID'].isin(sub['Question_ID'])]])
subzindi.to_csv('subllm.csv', index=False)